# SPDR/Index ETF Data Gathering and PreProcessing

<hr style="border: 4px solid royalblue">

### Disclaimer: This notebook should not be considered any kind of financial advice. It exists only for the purposes of practicing modeling and making predictions

In [2]:
from statsmodels.regression.rolling import RollingOLS
import yfinance as yf
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import pandas_ta
import statsmodels.api as sm
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 25)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Download Data
For the guide, we will be downloading data for the Dow Jones, S&P 500, Nasdaq, and Russell 2000. Not all date ranges have filled values for each index, and therefore their tickers will not appear in the data until at such time that we have that data from yfinance. This process would be the same if downloading data about other stocks or ETFs, you simply need to update the tickers in the cell below.

In [3]:
# Download Index Data from yfinance
sector_spdrs = ["XLB", "XLC", "XLY", "XLP", 'XLE', "XLF", 'XLV', "XLI", "XLK", "XLU", "XLRE"]
indeces = ["^RUT", "^IXIC", "^GSPC", "^DJI"]
index_df = yf.download(sector_spdrs, start='1964-01-02', end='2023-11-03')

[*********************100%%**********************]  11 of 11 completed


In [4]:
# Visual inspection, initially set up with multi-index columns, hence the .stack() method use
index_df = index_df.stack()
index_df.head(3)

Adj Close      Close       High        Low       Open   Volume
Date                                                                          
1998-12-22 XLB  12.011766  20.828125  20.828125  20.750000  20.781250   1900.0
           XLE  12.448166  23.265625  23.390625  23.187500  23.312500  15200.0
           XLF  11.612926  18.937855  19.052092  18.849005  19.052092  55887.0

In [5]:
# Reset the index to access the dates, create datetime
index_df.reset_index(inplace=True)

# convert to datetime
index_df["Date"] = pd.to_datetime(index_df["Date"])

# set the index
index_df.set_index("Date", inplace=True)

In [6]:
# Renaming ticker column
index_df.rename(columns={"level_1":"Ticker"}, inplace=True)
index_df.head(2)

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
1998-12-22,XLB,12.011766,20.828125,20.828125,20.7500,20.78125,1900.0
1998-12-22,XLE,12.448166,23.265625,23.390625,23.1875,23.31250,15200.0


# Data outside of yfinance

In [7]:
# Download historical GDP data from Federal Reserve Economic Data (F.R.E.D.)
gdp_data = web.get_data_fred("GDP", start="1964-01-01", end="2024-01-01")

# Download historical interest rates from the Federal Reserve
interest_rate_data = web.get_data_fred("DTB3", start="1964-01-02", end="2024-01-01")

# Download csv file from: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
ff = pd.read_csv('/Users/samalainabayeva/Desktop/Capstone Project/Dataset/F-F_Research_Data_5_Factors_2x3_daily.CSV', skiprows=2)

### Fama-French Preprocessing:

In [8]:
ff.head(2)

,Unnamed: 0,Mkt-RF,SMB,HML,RMW,CMA,RF
0,19630701,-0.67,0.02,-0.35,0.03,0.13,0.012
1,19630702,0.79,-0.28,0.28,-0.08,-0.21,0.012


In [9]:
# Converting Column to string for purposes of creating datetime
ff["Unnamed: 0"] = ff["Unnamed: 0"].astype(str)

# formating the string to match other datetime data
ff["Unnamed: 0"] = ff["Unnamed: 0"].apply(lambda x: f"{x[:4]}-{x[4:6]}-{x[6:]}")

# converting the properly formatted column to a datetime object in an appropriately named column
ff["Date"] = pd.to_datetime(ff["Unnamed: 0"])

# removing undesired features
ff.drop(columns=["Unnamed: 0", "RF"], inplace=True)

# Setting the datetime as index
ff.set_index("Date", inplace=True)

# Convert %'s into decimal values (might be unneccesary with future scaling)
ff = ff.div(100)

### GDP Preprocessing: a little tricky because of missing dates
GDP data has many dates missing, so I had to create an empty date range and merge it with the known data.

In [10]:
gdp_data.head(2)

,GDP
DATE,
1964-01-01,669.822
1964-04-01,678.674


In [11]:
# Create a date range spanning multiple years
start_date = pd.Timestamp(year=gdp_data.index.min().year, month=gdp_data.index.min().month, \
                          day=gdp_data.index.min().day)

# Known reasonable end date
end_date = pd.Timestamp(year=2023, month=7, day=1)

# Create the empty range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Merge the date range with the filtered GDP data, filling in missing dates
merged_data = date_range.to_frame().merge(gdp_data, how='left', left_on=date_range, right_index=True)

In [12]:
# Now the dates have been expanded to match our stock data dates
merged_data.head(3)

,0,GDP
1964-01-01,1964-01-01,669.822
1964-01-02,1964-01-02,NaN
1964-01-03,1964-01-03,NaN


In [13]:
# Filling in the missing dates using forward fill, while GDP is not static, it is also unknown and estimated between
# reporting periods, so forward filling seemed the best way to avoid exposing the data to future GDP, this way, the 
# data only has access to GDP after the moment it is reported quarterly. 
merged_data["GDP_Filled"] = merged_data["GDP"].ffill()
merged_data.drop(columns=[0, "GDP"], inplace=True)
merged_data.head(3)

,GDP_Filled
1964-01-01,669.822
1964-01-02,669.822
1964-01-03,669.822


In [14]:
ff.head(3)

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
1963-07-01,-0.0067,0.0002,-0.0035,0.0003,0.0013
1963-07-02,0.0079,-0.0028,0.0028,-0.0008,-0.0021
1963-07-03,0.0063,-0.0018,-0.0010,0.0013,-0.0025


### Interest rates

In [15]:
interest_rate_data.rename(columns={"DTB3": "Interest_Rates"}, inplace=True)
interest_rate_data.head(3)

,Interest_Rates
DATE,
1964-01-02,3.53
1964-01-03,3.53
1964-01-06,3.53


In [16]:
index_df.head(3)

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
1998-12-22,XLB,12.011766,20.828125,20.828125,20.750000,20.781250,1900.0
1998-12-22,XLE,12.448166,23.265625,23.390625,23.187500,23.312500,15200.0
1998-12-22,XLF,11.612926,18.937855,19.052092,18.849005,19.052092,55887.0


##### All of our data share date time indices, although they do not all date back to the same dates, therefore, it is important that we join them correctly to the data with the most restrictive index time range.

In [17]:
# inspecting dates and shape
index_df.index.min(), index_df.index.max(), index_df.shape

(Timestamp('1998-12-22 00:00:00'),
 Timestamp('2023-11-02 00:00:00'),
 (59699, 7))

In [18]:
# Joining the data together on the shared index values
df = index_df.join(merged_data).join(interest_rate_data).join(ff)
df.index.min(), df.index.max(), df.shape

(Timestamp('1998-12-22 00:00:00'),
 Timestamp('2023-11-02 00:00:00'),
 (59699, 14))

In [19]:
# Inspecting the data
df.head()

,Ticker,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,RMW,CMA
1998-12-22,XLB,12.011766,20.828125,20.828125,20.750000,20.781250,1900.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078
1998-12-22,XLE,12.448166,23.265625,23.390625,23.187500,23.312500,15200.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078
1998-12-22,XLF,11.612926,18.937855,19.052092,18.849005,19.052092,55887.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078
1998-12-22,XLI,14.971388,23.281250,23.281250,23.203125,23.203125,600.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078
1998-12-22,XLK,24.243130,32.046875,32.500000,31.781250,32.406250,300500.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078


# Technical Indicator Additions

In [21]:
df.head(2)

,index,Ticker,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,RMW,CMA
0,1998-12-22,XLB,12.011766,20.828125,20.828125,20.7500,20.78125,1900.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078
1,1998-12-22,XLE,12.448166,23.265625,23.390625,23.1875,23.31250,15200.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078


In [22]:
# re-structuring data, some of these may or may not be neccessary
# df.reset_index(inplace=True)
# df.drop(columns="level_0", inplace=True)
df.rename(columns={'index':"Date"}, inplace=True)
df.set_index(["Date",'Ticker'], inplace=True)

In [23]:
df.head(2)

Adj Close      Close       High      Low      Open  \
Date       Ticker                                                       
1998-12-22 XLB     12.011766  20.828125  20.828125  20.7500  20.78125   
           XLE     12.448166  23.265625  23.390625  23.1875  23.31250   

                    Volume  GDP_Filled  Interest_Rates  Mkt-RF     SMB  \
Date       Ticker                                                        
1998-12-22 XLB      1900.0    9293.991            4.39     0.0 -0.0029   
           XLE     15200.0    9293.991            4.39     0.0 -0.0029   

                      HML     RMW     CMA  
Date       Ticker                          
1998-12-22 XLB    -0.0043 -0.0009  0.0078  
           XLE    -0.0043 -0.0009  0.0078

### Garman-Klass Volatility

In [24]:
# Volatility Measure

df["garman_klass_vol"] = ((np.log(df['High']) - np.log(df["Low"])) ** 2)/2 - \
                        (2*np.log(2)- 1) * (np.log(df["Adj Close"])- np.log(df['Open']))**2

In [25]:
# Replacing the -inf values created by np.log(df['Open']) when early-dated open values == 0
# leaving value as nan for imputation later on
df["garman_klass_vol"].replace(-np.inf, np.nan, inplace=True)
df.head(3)

Adj Close      Close       High        Low       Open  \
Date       Ticker                                                          
1998-12-22 XLB     12.011766  20.828125  20.828125  20.750000  20.781250   
           XLE     12.448166  23.265625  23.390625  23.187500  23.312500   
           XLF     11.612926  18.937855  19.052092  18.849005  19.052092   

                    Volume  GDP_Filled  Interest_Rates  Mkt-RF     SMB  \
Date       Ticker                                                        
1998-12-22 XLB      1900.0    9293.991            4.39     0.0 -0.0029   
           XLE     15200.0    9293.991            4.39     0.0 -0.0029   
           XLF     55887.0    9293.991            4.39     0.0 -0.0029   

                      HML     RMW     CMA  garman_klass_vol  
Date       Ticker                                            
1998-12-22 XLB    -0.0043 -0.0009  0.0078         -0.116068  
           XLE    -0.0043 -0.0009  0.0078         -0.152027  
           XLF    -0.0043 -0.0009  0.0078         -0.094617

### pandas_ta methods -> technical analysis metrics

In [26]:
# Adding a feature for relative strength indicator -> takes 20 sessions to update

df["RSI"] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

In [27]:
df.head(2)

Adj Close      Close       High      Low      Open  \
Date       Ticker                                                       
1998-12-22 XLB     12.011766  20.828125  20.828125  20.7500  20.78125   
           XLE     12.448166  23.265625  23.390625  23.1875  23.31250   

                    Volume  GDP_Filled  Interest_Rates  Mkt-RF     SMB  \
Date       Ticker                                                        
1998-12-22 XLB      1900.0    9293.991            4.39     0.0 -0.0029   
           XLE     15200.0    9293.991            4.39     0.0 -0.0029   

                      HML     RMW     CMA  garman_klass_vol  RSI  
Date       Ticker                                                 
1998-12-22 XLB    -0.0043 -0.0009  0.0078         -0.116068  NaN  
           XLE    -0.0043 -0.0009  0.0078         -0.152027  NaN

In [28]:
# These also require the length of 20 sessions to show up

# 2 STD DEVs below 20-day SMA
df['lowest_bollinger_band'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                            close=x, length=20).iloc[:,0] if pandas_ta.bbands(close=x, length=20) \
                            is not None else np.nan)

# 1 STD DEV below 20-day SMA
df['lower_bollinger_band'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                            close=x, length=20, std=1).iloc[:,0] if pandas_ta.bbands(close=x, length=20) \
                            is not None else np.nan)

# 20 - Day Simple moving average

df['20_day_SMA'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                            close=x, length=20).iloc[:,1] if pandas_ta.bbands(close=x, length=20) \
                            is not None else np.nan)

# 1 STD DEV above SMA

df['one_up_bollinger_band'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                        close=x, length=20, std=1).iloc[:,2] if pandas_ta.bbands(close=x, length=20) \
                        is not None else np.nan)

# 2 STD DEVs above SMA

df['upper_bollinger_band'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                            close=x, length=20).iloc[:,2] if pandas_ta.bbands(close=x, length=20) \
                            is not None else np.nan)

In [29]:
df.head(1)

,,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,RMW,CMA,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band
Date,Ticker,,,,,,,,,,,,,,,,,,,,
1998-12-22,XLB,12.011766,20.828125,20.828125,20.75,20.78125,1900.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078,-0.116068,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Function for calculating the atr -> because multiple columns needed for tranformation, .transform() will not work
# because .transform() can only take 1 column as an input, we  will use a groupby().apply() function

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data["High"],
                 low=stock_data["Low"],
                 close=stock_data["Close"],
                       length=14)
    return atr

In [31]:
# Need to add group_keys = False, otherwise, it will double the date index, giving us a triple index

df["ATR"] = df.groupby(level=1, group_keys=False).apply(compute_atr)

In [32]:
df.head(1)

,,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,RMW,CMA,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band,ATR
Date,Ticker,,,,,,,,,,,,,,,,,,,,,
1998-12-22,XLB,12.011766,20.828125,20.828125,20.75,20.78125,1900.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078,-0.116068,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Custom function for adding a new feature, utilizing pandas_ta

def compute_MACD(close):
    macd_df = pandas_ta.macd(close=close, length=20)
    
    if macd_df is not None and not macd_df.empty:
        macd = macd_df.iloc[:, 0]
        return macd
    else:
        # This else clause was very tricky to figure out, required a series of nans in the proper length,
        # which was on occassion less than 20, therefore causing errors
        return pd.Series([np.nan] * len(close), index=close.index)

In [34]:
# Moving Average Convergence Divergence

df["MACD"] = df.groupby(level=1, group_keys=False)["Adj Close"].apply(compute_MACD)

In [35]:
df.head(1)

,,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,RMW,CMA,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band,ATR,MACD
Date,Ticker,,,,,,,,,,,,,,,,,,,,,,
1998-12-22,XLB,12.011766,20.828125,20.828125,20.75,20.78125,1900.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078,-0.116068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Dollar volume addition, amount of dollars worth of stock/etf/index traded in a day in millions

df["dollar_volume(M)"] = ((df['Adj Close']*df["Volume"])/1000000).round(4)

In [37]:
# Inspection
df.head()

Adj Close      Close       High        Low       Open  \
Date       Ticker                                                          
1998-12-22 XLB     12.011766  20.828125  20.828125  20.750000  20.781250   
           XLE     12.448166  23.265625  23.390625  23.187500  23.312500   
           XLF     11.612926  18.937855  19.052092  18.849005  19.052092   
           XLI     14.971388  23.281250  23.281250  23.203125  23.203125   
           XLK     24.243130  32.046875  32.500000  31.781250  32.406250   

                     Volume  GDP_Filled  Interest_Rates  Mkt-RF     SMB  \
Date       Ticker                                                         
1998-12-22 XLB       1900.0    9293.991            4.39     0.0 -0.0029   
           XLE      15200.0    9293.991            4.39     0.0 -0.0029   
           XLF      55887.0    9293.991            4.39     0.0 -0.0029   
           XLI        600.0    9293.991            4.39     0.0 -0.0029   
           XLK     300500.0    9293.991            4.39     0.0 -0.0029   

                      HML     RMW     CMA  garman_klass_vol  RSI  \
Date       Ticker                                                  
1998-12-22 XLB    -0.0043 -0.0009  0.0078         -0.116068  NaN   
           XLE    -0.0043 -0.0009  0.0078         -0.152027  NaN   
           XLF    -0.0043 -0.0009  0.0078         -0.094617  NaN   
           XLI    -0.0043 -0.0009  0.0078         -0.074152  NaN   
           XLK    -0.0043 -0.0009  0.0078         -0.032286  NaN   

                   lowest_bollinger_band  lower_bollinger_band  20_day_SMA  \
Date       Ticker                                                            
1998-12-22 XLB                       NaN                   NaN         NaN   
           XLE                       NaN                   NaN         NaN   
           XLF                       NaN                   NaN         NaN   
           XLI                       NaN                   NaN         NaN   
           XLK                       NaN                   NaN         NaN   

                   one_up_bollinger_band  upper_bollinger_band  ATR  MACD  \
Date       Ticker                                                           
1998-12-22 XLB                       NaN                   NaN  NaN   NaN   
           XLE                       NaN                   NaN  NaN   NaN   
           XLF                       NaN                   NaN  NaN   NaN   
           XLI                       NaN                   NaN  NaN   NaN   
           XLK                       NaN                   NaN  NaN   NaN   

                   dollar_volume(M)  
Date       Ticker                    
1998-12-22 XLB               0.0228  
           XLE               0.1892  
           XLF               0.6490  
           XLI               0.0090  
           XLK               7.2851

# Functions for target dates -> generate future dates and retrieve values

In [38]:
# This group of functions works -> Just replace the DF in the initial dates=<DF>["Date"]

def one_month_later(date, months=1):
    dates = df["Date"].unique()
    # Start with a specific date
    start_date = date 
    
    # Calculate the date one month later, adjusting for weekends
    one_month_later = start_date + pd.DateOffset(months=months)

    while one_month_later not in dates:  
        if one_month_later > dates[-1]:
            return np.nan
            
        else:
            one_month_later += pd.DateOffset(days=1)
    
    return one_month_later


def three_months_later(date, months=3):
    return one_month_later(date, months=months)


def six_months_later(date, months=6):
    return one_month_later(date, months=months)


def twelve_months_later(date, months=12):
    return one_month_later(date, months=months)



In [39]:
# Gaining access to Date-Times

df.reset_index(inplace=True)
df.head(2)

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,RMW,CMA,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band,ATR,MACD,dollar_volume(M)
0,1998-12-22,XLB,12.011766,20.828125,20.828125,20.7500,20.78125,1900.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078,-0.116068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0228
1,1998-12-22,XLE,12.448166,23.265625,23.390625,23.1875,23.31250,15200.0,9293.991,4.39,0.0,-0.0029,-0.0043,-0.0009,0.0078,-0.152027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1892


In [40]:
# Creating columns for quarter, month, day of week, year of presidential cycle, using string type for cat encoding

df["Quarter"] = df["Date"].dt.quarter.astype(str)
df["Month"] = df["Date"].dt.month.astype(str)
df["cycle_year"] = (df["Date"].dt.year % 4)
df["day_of_week"] = df["Date"].dt.day_of_week.astype(str)
df['cycle_year'] = df['cycle_year'].apply(lambda x: 4 if x == 0 else x).astype(str)
df['day_of_week'] = df["day_of_week"].map({'0':'Monday','1':'Tuesday','2':'Wednesday','3':'Thursday','4':'Friday'})

In [41]:
df.head(2)

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,...,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band,ATR,MACD,dollar_volume(M),Quarter,Month,cycle_year,day_of_week
0,1998-12-22,XLB,12.011766,20.828125,20.828125,20.7500,20.78125,1900.0,9293.991,4.39,0.0,-0.0029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0228,4,12,2,Tuesday
1,1998-12-22,XLE,12.448166,23.265625,23.390625,23.1875,23.31250,15200.0,9293.991,4.39,0.0,-0.0029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1892,4,12,2,Tuesday


In [42]:
# Cell takes about 3 minutes to run

df['one_months_later'] = df["Date"].apply(one_month_later)
df["three_months_later"] = df["Date"].apply(three_months_later)
df["six_months_later"] = df["Date"].apply(six_months_later)
df["twelve_months_later"] = df["Date"].apply(twelve_months_later)

In [43]:
# These functions add changes in percentage at the target dates, requires date to be index

def one_month_price_change(df, num="One"):
    for ticker in list(df["Ticker"].unique()):
        ticker_df = df[df["Ticker"] == ticker]

        for index, row in ticker_df.iterrows():
            try:
                one_month_later_value = row[f"{num.lower()}_months_later"]
                if not pd.isna(one_month_later_value):
                    ticker_df.at[index, f"{num}_Month_Change"] = (ticker_df.loc[one_month_later_value, "Adj Close"] - row["Adj Close"]) / row["Adj Close"]
                else:
                    # Handle the case where one_month_later is NaN (NaT)
                    ticker_df.at[index, f"{num}_Month_Change"] = np.nan
            except KeyError:
                # Handle the KeyError exception here if necessary
                ticker_df.at[index, f"{num}_Month_Change"] = np.nan
           
        df.loc[df["Ticker"] == ticker, f"{num}_Month_Change"] = ticker_df[f"{num}_Month_Change"]
    
    return df



def three_month_price_change(df, num="Three"):
    return one_month_price_change(df, num)
    
    
def six_month_price_change(df, num="Six"):
    return one_month_price_change(df, num)


def twelve_month_price_change(df, num="Twelve"):
    return one_month_price_change(df, num)

In [44]:
# resetting date to the index for use with the next set of functions
df.set_index("Date", inplace=True)

In [45]:
# adding changes in percentage for inferential purposes - takes a couple of minutes to run

one_month_price_change(df)
three_month_price_change(df)
six_month_price_change(df)
twelve_month_price_change(df)

,Ticker,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,...,Quarter,Month,cycle_year,day_of_week,one_months_later,three_months_later,six_months_later,twelve_months_later,One_Month_Change,Three_Month_Change,Six_Month_Change,Twelve_Month_Change
Date,,,,,,,,,,,,,,,,,,,,,,,,,
1998-12-22,XLB,12.011766,20.828125,20.828125,20.750000,20.781250,1900.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,4,12,2,Tuesday,1999-01-22,1999-03-22,1999-06-22,1999-12-22,0.012003,0.078635,0.256417,0.218479
1998-12-22,XLE,12.448166,23.265625,23.390625,23.187500,23.312500,15200.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,4,12,2,Tuesday,1999-01-22,1999-03-22,1999-06-22,1999-12-22,-0.028879,0.075714,0.209872,0.150694
1998-12-22,XLF,11.612926,18.937855,19.052092,18.849005,19.052092,55887.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,4,12,2,Tuesday,1999-01-22,1999-03-22,1999-06-22,1999-12-22,-0.009383,0.099426,0.099781,0.012503
1998-12-22,XLI,14.971388,23.281250,23.281250,23.203125,23.203125,600.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,4,12,2,Tuesday,1999-01-22,1999-03-22,1999-06-22,1999-12-22,0.010738,0.084570,0.272790,0.213058
1998-12-22,XLK,24.243130,32.046875,32.500000,31.781250,32.406250,300500.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,4,12,2,Tuesday,1999-01-22,1999-03-22,1999-06-22,1999-12-22,0.084349,0.107752,0.223306,0.655810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-02,XLP,68.669998,68.669998,68.769997,67.769997,68.029999,21175900.0,NaN,5.28,NaN,NaN,NaN,...,4,11,3,Thursday,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN
2023-11-02,XLRE,34.299999,34.299999,34.450001,33.740002,33.810001,8882200.0,NaN,5.28,NaN,NaN,NaN,...,4,11,3,Thursday,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN
2023-11-02,XLU,61.500000,61.500000,61.820000,60.400002,60.500000,25550100.0,NaN,5.28,NaN,NaN,NaN,...,4,11,3,Thursday,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN


In [46]:
# This function creates binary "No-Gain:0, Gain:1" columns

def one_month_gainer(df, num="One"):
    for ticker in list(df["Ticker"].unique()):
        ticker_df = df[df["Ticker"] == ticker]

        for index, row in ticker_df.iterrows():
            try:
                one_month_later_value = row[f"{num.lower()}_months_later"]
                if not pd.isna(one_month_later_value):
                    ticker_df.at[index, f"{num}_Month_Positive"] = (row["Adj Close"] < ticker_df.loc[\
                                                                    one_month_later_value, "Adj Close"]).astype(int)
                else:
                    # Handle the case where one_month_later is NaN (NaT)
                    ticker_df.at[index, f"{num}_Month_Positive"] = np.nan
            except KeyError:
                # Handle the KeyError exception here if necessary
                ticker_df.at[index, f"{num}_Month_Positive"] = np.nan
           
        df.loc[df["Ticker"] == ticker, f"{num}_Month_Positive"] = ticker_df[f"{num}_Month_Positive"]
    
    return df


def three_month_gainer(df, num="Three"):
    return one_month_gainer(df, num)


def six_month_gainer(df, num="Six"):
    return one_month_gainer(df, num)


def twelve_month_gainer(df, num="Twelve"):
    return one_month_gainer(df, num)

In [47]:
# Adding the binary columns which will be our target
one_month_gainer(df)
three_month_gainer(df)
six_month_gainer(df)
twelve_month_gainer(df)

,Ticker,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,...,one_months_later,three_months_later,six_months_later,twelve_months_later,One_Month_Change,Three_Month_Change,Six_Month_Change,Twelve_Month_Change,One_Month_Positive,Three_Month_Positive,Six_Month_Positive,Twelve_Month_Positive
Date,,,,,,,,,,,,,,,,,,,,,,,,,
1998-12-22,XLB,12.011766,20.828125,20.828125,20.750000,20.781250,1900.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,1999-01-22,1999-03-22,1999-06-22,1999-12-22,0.012003,0.078635,0.256417,0.218479,1.0,1.0,1.0,1.0
1998-12-22,XLE,12.448166,23.265625,23.390625,23.187500,23.312500,15200.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,1999-01-22,1999-03-22,1999-06-22,1999-12-22,-0.028879,0.075714,0.209872,0.150694,0.0,1.0,1.0,1.0
1998-12-22,XLF,11.612926,18.937855,19.052092,18.849005,19.052092,55887.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,1999-01-22,1999-03-22,1999-06-22,1999-12-22,-0.009383,0.099426,0.099781,0.012503,0.0,1.0,1.0,1.0
1998-12-22,XLI,14.971388,23.281250,23.281250,23.203125,23.203125,600.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,1999-01-22,1999-03-22,1999-06-22,1999-12-22,0.010738,0.084570,0.272790,0.213058,1.0,1.0,1.0,1.0
1998-12-22,XLK,24.243130,32.046875,32.500000,31.781250,32.406250,300500.0,9293.991,4.39,0.0,-0.0029,-0.0043,...,1999-01-22,1999-03-22,1999-06-22,1999-12-22,0.084349,0.107752,0.223306,0.655810,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-02,XLP,68.669998,68.669998,68.769997,67.769997,68.029999,21175900.0,NaN,5.28,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-02,XLRE,34.299999,34.299999,34.450001,33.740002,33.810001,8882200.0,NaN,5.28,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-02,XLU,61.500000,61.500000,61.820000,60.400002,60.500000,25550100.0,NaN,5.28,NaN,NaN,NaN,...,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Handling nulls
If you so choose to keep more data, you can handle these null values differently. My intention was to model over different time frames, and so the early/late data contained nulls due to requiring x-number of periods to generate technical indicators, and x-days in the future in order to know whether or not the price increased from a given date. I therefore dropped all non-garman_klass nulls

In [52]:
df.shape

(59699, 40)

In [53]:
df.isna().sum()

Ticker                      0
Adj Close                   0
Close                       0
High                        0
Low                         0
Open                        0
Volume                      0
GDP_Filled                957
Interest_Rates            441
Mkt-RF                    737
SMB                       737
HML                       737
RMW                       737
CMA                       737
garman_klass_vol            0
RSI                       220
lowest_bollinger_band     209
lower_bollinger_band      209
20_day_SMA                209
one_up_bollinger_band     209
upper_bollinger_band      209
ATR                       154
MACD                      275
dollar_volume(M)            0
Quarter                     0
Month                       0
cycle_year                  0
day_of_week                 0
one_months_later          253
three_months_later        715
six_months_later         1408
twelve_months_later      2761
One_Month_Change          253
Three_Mont

In [54]:
df.dropna(subset=df.columns.difference(["garman_klass_vol"]), inplace=True)

In [55]:
df.isna().sum()

Ticker                   0
Adj Close                0
Close                    0
High                     0
Low                      0
Open                     0
Volume                   0
GDP_Filled               0
Interest_Rates           0
Mkt-RF                   0
SMB                      0
HML                      0
RMW                      0
CMA                      0
garman_klass_vol         0
RSI                      0
lowest_bollinger_band    0
lower_bollinger_band     0
20_day_SMA               0
one_up_bollinger_band    0
upper_bollinger_band     0
ATR                      0
MACD                     0
dollar_volume(M)         0
Quarter                  0
Month                    0
cycle_year               0
day_of_week              0
one_months_later         0
three_months_later       0
six_months_later         0
twelve_months_later      0
One_Month_Change         0
Three_Month_Change       0
Six_Month_Change         0
Twelve_Month_Change      0
One_Month_Positive       0
T

In [56]:
# Inspection
df["One_Month_Positive"].value_counts(), df["Three_Month_Positive"].value_counts(),\
df["Six_Month_Positive"].value_counts(), df["Twelve_Month_Positive"].value_counts()

(One_Month_Positive
 1.0    34054
 0.0    22192
 Name: count, dtype: int64,
 Three_Month_Positive
 1.0    36849
 0.0    19397
 Name: count, dtype: int64,
 Six_Month_Positive
 1.0    38373
 0.0    17873
 Name: count, dtype: int64,
 Twelve_Month_Positive
 1.0    41047
 0.0    15199
 Name: count, dtype: int64)

In [57]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 56246 entries, 1999-01-29 to 2022-11-02
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Ticker                 56246 non-null  object        
 1   Adj Close              56246 non-null  float64       
 2   Close                  56246 non-null  float64       
 3   High                   56246 non-null  float64       
 4   Low                    56246 non-null  float64       
 5   Open                   56246 non-null  float64       
 6   Volume                 56246 non-null  float64       
 7   GDP_Filled             56246 non-null  float64       
 8   Interest_Rates         56246 non-null  float64       
 9   Mkt-RF                 56246 non-null  float64       
 10  SMB                    56246 non-null  float64       
 11  HML                    56246 non-null  float64       
 12  RMW                    56246 non-null  floa

# Final Data
Next Steps would be to add more technical indicators or add more commodity prices, etc.

In [58]:
df

,Ticker,Adj Close,Close,High,Low,Open,Volume,GDP_Filled,Interest_Rates,Mkt-RF,SMB,HML,...,one_months_later,three_months_later,six_months_later,twelve_months_later,One_Month_Change,Three_Month_Change,Six_Month_Change,Twelve_Month_Change,One_Month_Positive,Three_Month_Positive,Six_Month_Positive,Twelve_Month_Positive
Date,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-29,XLB,12.173973,21.109375,21.109375,20.953125,20.953125,400.0,9411.682,4.37,0.0095,0.0010,-0.0062,...,1999-03-01,1999-04-29,1999-07-29,2000-01-31,0.003700,0.304891,0.224777,0.120443,1.0,1.0,1.0,1.0
1999-01-29,XLE,11.670672,21.812500,21.812500,21.437500,21.687500,18600.0,9411.682,4.37,0.0095,0.0010,-0.0062,...,1999-03-01,1999-04-29,1999-07-29,2000-01-31,-0.018624,0.321353,0.294086,0.272486,0.0,1.0,1.0,1.0
1999-01-29,XLF,11.877566,19.369415,19.394800,18.810926,19.077478,72998.0,9411.682,4.37,0.0095,0.0010,-0.0062,...,1999-03-01,1999-04-29,1999-07-29,2000-01-31,0.019659,0.138006,0.056829,-0.020673,1.0,1.0,1.0,0.0
1999-01-29,XLI,15.624509,24.296875,24.406250,23.953125,24.234375,109200.0,9411.682,4.37,0.0095,0.0010,-0.0062,...,1999-03-01,1999-04-29,1999-07-29,2000-01-31,0.008360,0.199315,0.202136,0.131147,1.0,1.0,1.0,1.0
1999-01-29,XLK,28.604773,37.812500,37.843750,36.937500,37.375000,933600.0,9411.682,4.37,0.0095,0.0010,-0.0062,...,1999-03-01,1999-04-29,1999-07-29,2000-01-31,-0.103306,-0.018595,0.069421,0.337216,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-02,XLP,69.437714,71.250000,73.080002,71.209999,72.360001,16291600.0,26408.405,4.04,-0.0267,-0.0087,0.0161,...,2022-12-02,2023-02-02,2023-05-02,2023-11-02,0.083368,0.041662,0.097803,-0.011056,1.0,1.0,1.0,0.0
2022-11-02,XLRE,34.450985,35.709999,36.950001,35.645000,36.470001,13999400.0,26408.405,4.04,-0.0267,-0.0087,0.0161,...,2022-12-02,2023-02-02,2023-05-02,2023-11-02,0.091291,0.184681,0.048500,-0.004383,1.0,1.0,1.0,0.0
2022-11-02,XLU,64.326530,66.389999,68.209999,66.360001,66.820000,21340200.0,26408.405,4.04,-0.0267,-0.0087,0.0161,...,2022-12-02,2023-02-02,2023-05-02,2023-11-02,0.069890,0.053284,0.044220,-0.043940,1.0,1.0,1.0,0.0


In [59]:
# Cell for creating different files for use

# df.to_csv("/Users/samalainabayeva/Desktop/Capstone Project/INDICES_FILLED.csv")
# df.to_csv("/Users/samalainabayeva/Desktop/Capstone Project/INDECES_w_technical_indicators.csv")
# df.to_csv('/Users/samalainabayeva/Desktop/Capstone Project/Indices_back_to_1964_with_indicators.csv')
df.to_csv('/Users/samalainabayeva/Desktop/Capstone Project/SPDR_DATA_W_ADDED.csv')